In [14]:
from citylearn import CityLearn
from pathlib import Path
import numpy as np                                                                                                                                                                                      

import ray
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.fcnet import FullyConnectedNetwork as TorchFC
from ray.rllib.models import ModelCatalog
from ray.rllib.agents import ppo
from ray.tune.logger import pretty_print
from ray.rllib.utils.framework import try_import_torch

torch, nn = try_import_torch()
ray.init(local_mode=True)

2021-11-27 22:12:15,089	ERROR import_thread.py:88 -- ImportThread: [WinError 10054] An existing connection was forcibly closed by the remote host


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [15]:
# Load environment
climate_zone = 5
params = {'data_path':Path("data/Climate_Zone_"+str(climate_zone)), 
        'building_attributes':'building_attributes.json', 
        'weather_file':'weather_data.csv', 
        'solar_profile':'solar_generation_1kW.csv', 
        'carbon_intensity':'carbon_intensity.csv',
        'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
        'buildings_states_actions':'buildings_state_action_space.json', 
        'simulation_period': (0, 8760*4-1), 
        'cost_function': ['ramping','1-load_factor','average_daily_peak','peak_demand','net_electricity_consumption','carbon_emissions'], 
        'central_agent': True,
        'save_memory': False }

# Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
env = CityLearn(**params)
observations_spaces, actions_spaces = env.get_state_action_spaces()

# Provides information on Building type, Climate Zone, Annual DHW demand, Annual Cooling Demand, Annual Electricity Demand, Solar Capacity, and correllations among buildings
building_info = env.get_building_information()

In [16]:
params_agent = {'building_ids':["Building_"+str(i) for i in [1,2,3,4,5,6,7,8,9]],
                'buildings_states_actions':'buildings_state_action_space.json', 
                'building_info':building_info,
                'observation_spaces':observations_spaces, 
                'action_spaces':actions_spaces}

In [17]:
class TorchCustomModel(TorchModelV2, nn.Module):
    """Example of a PyTorch custom model that just delegates to a fc-net."""

    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs,
                              model_config, name)
        nn.Module.__init__(self)

        self.torch_sub_model = TorchFC(obs_space, action_space, num_outputs,
                                       model_config, name)

    def forward(self, input_dict, state, seq_lens):
        input_dict["obs"] = input_dict["obs"].float()
        fc_out, _ = self.torch_sub_model(input_dict, state, seq_lens)
        return fc_out, []

    def value_function(self):
        return torch.reshape(self.torch_sub_model.value_function(), [-1])

In [18]:
ModelCatalog.register_custom_model('my_model', TorchCustomModel)

In [19]:
config = {
    'env': CityLearn,
    'env_config': params,
    'num_gpus': 1,
    'model': {
        'custom_model': 'my_model',
        'vf_share_layers': True,
        'custom_model_config': params_agent
    },
    'num_workers': 1,
}

In [ ]:
ppo_config = ppo.DEFAULT_CONFIG.copy()
ppo_config.update(config)
ppo_config['lr'] = 1e-3
trainer = ppo.PPOTrainer(config=ppo_config, env=CityLearn)
result = trainer.train()
print(pretty_print(result))

In [13]:
ray.shutdown()

In [ ]:


# Instantiating the control agent(s)
agents = Agent(**params_agent)

In [ ]:
state = env.reset()
done = False

action, coordination_vars = agents.select_action(state)    
while not done:
    next_state, reward, done, _ = env.step(action)
    action_next, coordination_vars_next = agents.select_action(next_state)
    agents.add_to_buffer(state, action, reward, next_state, done, coordination_vars, coordination_vars_next)
    coordination_vars = coordination_vars_next
    state = next_state
    action = action_next

env.cost()